Processing c:\private\var\folders\k1\30mswbxs7r1g6zwn8y4fyt500000gp\t\abs_a17a7759g2\croot\anyio_1706220182417\work (from -r C:/Users/LO79RS/topquartile/requirements.txt (line 2))
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\private\\var\\folders\\k1\\30mswbxs7r1g6zwn8y4fyt500000gp\\T\\abs_a17a7759g2\\croot\\anyio_1706220182417\\work'



In [1]:
import sys
from pathlib import Path

root = Path().resolve()
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print("Folder:", root)

Folder: C:\Users\LO79RS\topquartile


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


In [3]:
from topquartile.modules.datamodule.dataloader import DataLoader
from topquartile.modules.datamodule.transforms.covariate import TechnicalCovariateTransform
from topquartile.modules.datamodule.transforms.label import BinaryLabelTransform
from topquartile.modules.datamodule.partitions import PurgedTimeSeriesPartition

ImportError: cannot import name 'BasePurgedTimeSeriesPartition' from 'topquartile.modules.datamodule.partitions' (C:\Users\LO79RS\topquartile\topquartile\modules\datamodule\partitions.py)

In [12]:
echo %cd%

c:\Users\LO79RS\topquartile


In [23]:
pip install quantile-forest

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for quantile-forest from https://files.pythonhosted.org/packages/c2/e3/03f4cdb528045d8514abcfe99eee615fc90035139c8328831847084766c2/quantile_forest-1.4.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scikit-learn>=1.5 from https://files.pythonhosted.org/packages/a1/a6/c5b78606743a1f28eae8f11973de6613a5ee87366796583fb74c67d54939/scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b31fea68726a4ae5f2d82ddd9/threadpoolctl-3.6.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/279.6 kB ? eta -:--:--
   ----- --------------------------------- 41.0/279.6 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------  276.5/279.6 kB 3.4 MB/s eta 0:00:01
   -----------------------

In [24]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for wandb from https://files.pythonhosted.org/packages/36/d5/215cac3edec5c5ac6e7231beb9d22466d5d4e4a132fa3a1d044f7d682c15/wandb-0.19.11-py3-none-win_amd64.whl.metadata
  Obtaining dependency information for docker-pycreds>=0.4.0 from https://files.pythonhosted.org/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gitpython!=3.1.29,>=1.0.0 from https://files.pythonhosted.org/packages/1d/9a/4114a9057db2f1462d5c8f8390ab7383925fe1ac012eaa42402ad65c2963/GitPython-3.1.44-py3-none-any.whl.metadata
  Obtaining dependency information for sentry-sdk>=2.0.0 from https://files.pythonhosted.org/packages/f0/e5/da07b0bd832cefd52d16f2b9bbbe31624d57552602c06631686b93ccb1bd/sentry_sdk-2.29.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for setproctitle from h